---

### 📜 Licence d'utilisation

Ce document est protégé sous licence **Creative Commons BY-NC-ND 4.0 International**  
🔒 **Aucune modification ni réutilisation sans autorisation explicite de l'auteur.**

- 👤 Auteur : Christie Vassilian  
- 📥 Téléchargement autorisé uniquement à usage pédagogique personnel  
- 🚫 Réutilisation commerciale ou modification interdite  

[![Licence CC BY-NC-ND](https://licensebuttons.net/l/by-nc-nd/4.0/88x31.png)](https://creativecommons.org/licenses/by-nc-nd/4.0/)

---

# NBJ 4 — Expliquer un modèle sans le connaître : LIME & SHAP
### Méthodes agnostiques + détection & correction de biais

Dans ce dernier TP, nous allons :
- expliquer une image sans aucune connaissance du modèle utilisé ;
- découvrir deux méthodes d'explicabilité **agnostiques** :
  - **LIME-Image** : segmentation en superpixels + modèle local ;
  - **SHAP (KernelSHAP)** : valeurs de Shapley appliquées à l'image ;
- détecter un biais présent dans un modèle (ex : main, arrière-plan, objet secondaire) ;
- corriger ce biais par des techniques simples ;
- comparer toutes les explications obtenues pendant les 4 TP.

Objectif : comprendre pourquoi l’explicabilité est essentielle pour la confiance.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import torch
from torchvision import models, transforms

from lime import lime_image
import shap
from skimage.segmentation import mark_boundaries

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT).to(device)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
         [0.485, 0.456, 0.406],
         [0.229, 0.224, 0.225]
    )
])

In [ ]:
# TODO : ajouter une image avec biais de contexte (ex : oiseau sur branche, objet dans une main)
img_path = "oiseau_branche.jpg"  # à adapter

img = Image.open(img_path).convert("RGB")
img_np = np.array(img)

plt.imshow(img_np)
plt.axis("off")
plt.title("Image originale")

### ❓ Questions 1–2
1. Sur cette image, quels éléments pourraient **détourner l'attention** du modèle ?  
2. Le contexte (branche, main, arrière-plan) peut-il influencer une IA ?

In [ ]:
def predict(images):
    tensors = []
    for img_np in images:
        img_pil = Image.fromarray(img_np.astype("uint8"))
        tensors.append(transform(img_pil))
    batch = torch.stack(tensors).to(device)
    with torch.no_grad():
        preds = model(batch)
        probs = torch.nn.functional.softmax(preds, dim=1)
    return probs.cpu().numpy()

In [ ]:
explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(
    img_np,
    classifier_fn=predict,
    top_labels=1,
    hide_color=0,
    num_samples=1000
)

In [ ]:
temp, mask = explanation.get_image_and_mask(
    label=explanation.top_labels[0],
    positive_only=True,
    num_features=5,
    hide_rest=False,
)

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.imshow(img_np)
plt.title("Image originale")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(mark_boundaries(temp, mask))
plt.title("LIME : superpixels importants")
plt.axis("off")
plt.tight_layout()

### ❓ Questions 3–6  
3. Que représente chaque superpixel ?  
4. LIME met-il en avant l’objet (oiseau) ou son contexte (branche) ?  
5. Est-ce cohérent avec ce qu’on *attend* d’une IA ?  
6. Si le modèle regarde la branche, quel type de biais cela révèle-t-il ?

In [ ]:
# Baseline très simple : une petite collection d'images identiques
background = np.array([img_np for _ in range(10)])

In [ ]:
explainer_shap = shap.KernelExplainer(predict, background)
shap_values = explainer_shap.shap_values(np.array([img_np]), nsamples=200)

In [ ]:
shap_img = shap_values[0][0].mean(axis=2)

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.imshow(img_np)
plt.title("Image originale")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(shap_img, cmap='inferno')
plt.title("SHAP : importance des pixels")
plt.axis("off")
plt.tight_layout()

### ❓ Questions 7–10  
7. SHAP montre-t-il (comme LIME) que le modèle utilise le contexte ?  
8. SHAP donne une importance **positive/négative** : comment interpréter cela ?  
9. SHAP te paraît-il plus “fin” que LIME ? Pourquoi ?  
10. Si le modèle se trompe, SHAP peut-il aider à comprendre *pourquoi* ?

# Comparaison LIME / SHAP / Grad-CAM / IG

- **LIME** : segmentation en superpixels → explication simple, locale  
- **SHAP** : valeurs de Shapley → importance positive/négative  
- **Grad-CAM** : zones générales, semi-globale (TP2)  
- **IG** : explication fine et stable (TP3)

### ❓ Questions 11–12  
11. Parmi les 4 méthodes, laquelle explique le mieux **le biais** de cette image ?  
12. Pourquoi est-il utile de combiner plusieurs explications ?

In [ ]:
# Exemple simple : recadrage (correction partielle du biais)
h, w, _ = img_np.shape
cropped = img_np[h//6:5*h//6, w//6:5*w//6]  # recadrage central (à ajuster selon l'image)

plt.imshow(cropped)
plt.axis("off")
plt.title("Image recadrée (bias réduit)")

In [ ]:
cropped_pred = predict([cropped])
cropped_pred

### ❓ Questions 13–15  
13. Après correction, le modèle regarde-t-il enfin le bon objet ?  
14. Le biais était-il lié **à l’image** ou **au modèle** ?  
15. Comment pourrait-on corriger ce biais de manière définitive (données, entraînement…) ?

# 💡 Synthèse finale — Pourquoi LIME & SHAP sont essentiels

- On peut expliquer un modèle **sans le connaître** → méthodes agnostiques.  
- LIME montre les parties essentielles d’une image (superpixels).  
- SHAP donne l’importance positive/négative des pixels.  
- Ensemble, elles permettent de :
  - détecter des **biais contextuels** ;
  - comprendre les erreurs du modèle ;
  - corriger et améliorer les données d’entraînement.

### Dernière question  
👉 Selon toi, quelle méthode (LIME, SHAP, IG, Grad-CAM, Occlusion…) te semble **la plus convaincante** pour comprendre une IA ?

---

### 📜 Licence d'utilisation

Ce document est protégé sous licence **Creative Commons BY-NC-ND 4.0 International**  
🔒 **Aucune modification ni réutilisation sans autorisation explicite de l'auteur.**

- 👤 Auteur : Christie Vassilian  
- 📥 Téléchargement autorisé uniquement à usage pédagogique personnel  
- 🚫 Réutilisation commerciale ou modification interdite  

[![Licence CC BY-NC-ND](https://licensebuttons.net/l/by-nc-nd/4.0/88x31.png)](https://creativecommons.org/licenses/by-nc-nd/4.0/)

---